# IR-1061 spectrum processing

## Preface

It is the first notebook devoted to demonstration of Python ecosystem applied to some tasks done before in MATLAB, Octave and even Fortran.

## IR-1061 desorption experiment

Some time ago our lab did a series of experiments on desorbing organic dyes intended for use as Q-switching IR laser dye as matrix for MALDI-like mass spectrometry of organic compounds.

https://www.researchgate.net/publication/258108090_Polyatomic_ions_mass_analysis_using_compact_laser_desorption_TOF_MS

IR-1061 also known as 4-[2-[2-Chloro-3-[(2,6-diphenyl-4H-thiopyran-4-ylidene)ethylidene]-1-cyclohexen-1-yl]ethenyl]-2,6-diphenylthiopyrylium tetrafluoroborate was purchased from Sigma-Aldrich.

CAS Number 155614-01-0

Empirical Formula (Hill Notation) C44H34ClS2 · BF4

Molecular Weight 749.13  

A fancy picture of the substance:

<img src="https://www.sigmaaldrich.com/content/dam/sigma-aldrich/structure0/133/mfcd00191915.eps/_jcr_content/renditions/mfcd00191915-medium.png" />

## Experimental setup

Absorption $\lambda_{max}$ = 1061 nm (in methylene chloride) makes it possible to easily desorb the substance with 1064 nm Nd:YAG laser.

Presented example of mass spectrum was acquired using custom time-of-flight laser desorption mass spectrometer. Digitized by Agilent DSO9104A. Let us look at obtained spectrum...

## Data Processing

In [1]:
import numpy as np
import scipy as sp

In [2]:
with open("IR-1061.txt") as SpectrumFile:
    data = np.loadtxt(SpectrumFile)
    time = data[1:8000, 0] * 1E6        #time in microseconds
    intensity = (-1)*data[1:8000, 1]    #invert

### Imported spectrum plot

In [3]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Range1d
#from bokeh.models import HoverTool

output_notebook()

Loading BokehJS ...

In [4]:
p = figure(title="IR-1061 mass spectrum", plot_height=200, plot_width=900)
p.xaxis.axis_label = 'Time, us'
p.yaxis.axis_label = 'Relative intensity'
p.x_range = Range1d(0, max(time), bounds=(0, max(time)))

p.line(time, intensity)

show(p)

### Baseline subtraction and ROI

We will do baseline correction just subtracting the median value. Also we will define a particular region of interest (ROI) - 4000 samples.

In [5]:
ROI = 4000
intensity = intensity[1:ROI]
time = time[1:ROI]
intensity = intensity-np.median(intensity)


p = figure(title="IR-1061 mass spectrum baseline corrected", plot_height=200, plot_width=900)
p.xaxis.axis_label = 'Time, us'
p.yaxis.axis_label = 'Relative intensity'
p.x_range = Range1d(0, max(time), bounds=(0, max(time)))
p.y_range = Range1d(-0.05, 1.05)

p.line(time, intensity)

show(p)

### Time to mass scale calibration

We should figure out the masses of ions having the time of arrival to the microchannel plate detector. The most simple formula for time-to-mass scale calibration is:
$$m = a(t+b)^2,$$

$$a = \dfrac{m}{(t+b)^2}$$.

Obviously we need 2 pairs of (m,t) to derive a and b. But we can estimate b as delay between trigger pulse and second ringing at the beginning of the spectrum. Do not ask why - it's a kind of scientific black magic.

### Infrared direct LDI and UV MALDI spectra comparison

Thanks to our friends and partners we got the opportunity to take some UV MALDI spectra of IR-1061 with commonly used CHCA matrix (α-Cyano-4-hydroxycinnamic acid)

### Time to mass scale refinement

### Spectrum smoothing, peak fitting and resolution calculation

We should perform some filtering to smooth the line preserving peak amplitude as much as it possible.

In [6]:
#   filtered = signal.savgol_filter(intensity, 25, 7)